In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suggestions/Market_Basket_Optimisation.csv
/kaggle/input/imdb-top-250-movies/IMDB_Top250_Movies.csv
/kaggle/input/movielens/movies.csv
/kaggle/input/movielens/ratings.csv
/kaggle/input/movielens/README.txt
/kaggle/input/movielens/tags.csv
/kaggle/input/movielens/links.csv


In [2]:
rating_data= pd.read_csv('../input/movielens/ratings.csv')
movies_names= pd.read_csv('../input/movielens/movies.csv')

In [3]:
# mereging two data frames
movie_data= pd.merge(rating_data, movies_names, on='movieId') # on is the common column in both which becomes link to join the 2 datasets

In [4]:
movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama


In [5]:
# popularity recomendation
movie_data.groupby('title')['rating'].mean().head() # ()is grouping title []is showing there rating column

title
"Great Performances" Cats (1998)           1.750000
$9.99 (2008)                               3.833333
'Hellboy': The Seeds of Creation (2004)    2.000000
'Neath the Arizona Skies (1934)            0.500000
'Round Midnight (1986)                     2.250000
Name: rating, dtype: float64

In [6]:
movie_data.groupby('title')['rating'].count().sort_values(ascending= False).head()

title
Forrest Gump (1994)                          341
Pulp Fiction (1994)                          324
Shawshank Redemption, The (1994)             311
Silence of the Lambs, The (1991)             304
Star Wars: Episode IV - A New Hope (1977)    291
Name: rating, dtype: int64

In [7]:
# forming data frame [it helps in forming report neatly]
ratings_mean_count= pd.DataFrame(movie_data.groupby('title')['rating'].mean())

In [8]:
ratings_mean_count['rating_counts']= pd.DataFrame(movie_data.groupby('title')['rating'].count())

In [9]:
ratings_mean_count.head()

,rating,rating_counts
title,,
"""Great Performances"" Cats (1998)",1.750000,2
$9.99 (2008),3.833333,3
'Hellboy': The Seeds of Creation (2004),2.000000,1
'Neath the Arizona Skies (1934),0.500000,1
'Round Midnight (1986),2.250000,2


The movies with 2 or more ratings will be recomended 

In [10]:
# content based recomandation
imdb_rating= pd.read_csv('../input/imdb-top-250-movies/IMDB_Top250_Movies.csv')

imdb_rating.shape

(250, 38)

In [11]:
imdb_rating.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [12]:
# extracting only select columns 
data1= imdb_rating[['Title', 'Genre', 'Director', 'Actors', 'Plot']]

In [13]:
data1['Plot']

0      Two imprisoned men bond over a number of years...
1      The aging patriarch of an organized crime dyna...
2      The early life and career of Vito Corleone in ...
3      When the menace known as the Joker emerges fro...
4      A jury holdout attempts to prevent a miscarria...
                             ...                        
245    The desperate life of a chronic alcoholic is f...
246    A 20-something supervising staff member of a r...
247    A newspaper editor uses every trick in the boo...
248    An old man makes a long journey by lawn-mover ...
249    A Mumbai teen reflects on his upbringing in th...
Name: Plot, Length: 250, dtype: object

In [14]:
# we can see in Actors column that the Actors name have "," in them 
# one of the nlp pre processing we will remove the "," and that 1st 3 names
data1['Actors']= data1['Actors'].map(lambda x:x.split(',')[:3])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
data1['Actors']

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [16]:
# same with Genre

data1['Genre']= data1['Genre'].map(lambda x:x.lower().split(','))
# map func map(function[here lambda is used], the itereator[here data['Genre']]) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# with director

data1['Director']= data1['Director'].map(lambda x:x.split(' '))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
data1['Director']

0                      [Frank, Darabont]
1               [Francis, Ford, Coppola]
2               [Francis, Ford, Coppola]
3                   [Christopher, Nolan]
4                        [Sidney, Lumet]
                     ...                
245                      [Billy, Wilder]
246            [Destin, Daniel, Cretton]
247                      [Howard, Hawks]
248                       [David, Lynch]
249    [Danny, Boyle,, Loveleen, Tandan]
Name: Director, Length: 250, dtype: object

In [19]:
for i,row in data1.iterrows():
    row['Actors']= [x.lower().replace(' ','') for x in row['Actors']]
    row['Director']= ''.join(row['Director']).lower()

In [20]:
data1['Actors']

0                 [timrobbins, morganfreeman, bobgunton]
1                    [marlonbrando, alpacino, jamescaan]
2                  [alpacino, robertduvall, dianekeaton]
3             [christianbale, heathledger, aaroneckhart]
4                 [martinbalsam, johnfiedler, leej.cobb]
                             ...                        
245                [raymilland, janewyman, phillipterry]
246     [brielarson, johngallagherjr., stephaniebeatriz]
247           [carygrant, rosalindrussell, ralphbellamy]
248    [sissyspacek, janegallowayheitz, josepha.carpe...
249                [devpatel, saurabhshukla, anilkapoor]
Name: Actors, Length: 250, dtype: object

In [21]:
data1['Director']

0                  frankdarabont
1             francisfordcoppola
2             francisfordcoppola
3               christophernolan
4                    sidneylumet
                 ...            
245                  billywilder
246          destindanielcretton
247                  howardhawks
248                   davidlynch
249    dannyboyle,loveleentandan
Name: Director, Length: 250, dtype: object

In [22]:
# Count vecotrization techinque breaking down a sentence or 
# any text into words by performing preprocessing tasks like converting all words to lowercase, thus removing special characters


In [23]:
pip install rake-nltk

Note: you may need to restart the kernel to use updated packages.


In [24]:
# mremoving stop words(stop words are words which occur ferequetnly like a,the,is,up, over,etc)
# once extract we can put it in dictionary
from rake_nltk import Rake

In [25]:
# creating new column to data frame
data1['keywords']= ""
for i,row in data1.iterrows():
    plot= row['Plot']
    r= Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores= r.get_word_degrees()
    row['keywords']= list(key_words_dict_scores.keys())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [27]:
data1['keywords']

0      [two, imprisoned, men, bond, number, years, fi...
1      [aging, patriarch, organized, crime, dynasty, ...
2      [early, life, career, vito, corleone, 1920s, n...
3      [menace, known, joker, emerges, mysterious, pa...
4      [jury, holdout, attempts, prevent, miscarriage...
                             ...                        
245    [desperate, life, chronic, alcoholic, followed...
246    [20, something, supervising, staff, member, re...
247    [newspaper, editor, uses, every, trick, book, ...
248    [old, man, makes, long, journey, lawn, mover, ...
249    [mumbai, teen, reflects, upbringing, slums, ac...
Name: keywords, Length: 250, dtype: object

In [28]:
data1.set_index('Title', inplace= True)
data1.head()

,Genre,Director,Actors,Plot,keywords
Title,,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]",Two imprisoned men bond over a number of years...,"[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]",The aging patriarch of an organized crime dyna...,"[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]",The early life and career of Vito Corleone in ...,"[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]",When the menace known as the Joker emerges fro...,"[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]",A jury holdout attempts to prevent a miscarria...,"[jury, holdout, attempts, prevent, miscarriage..."


In [29]:
# creating column for bag of words
data1['bag_of_words']=""
columns= data1.columns
for index, row in data1.iterrows():
    words=""
    for col in columns:
          if col != 'Director':
            words= words+' '.join(row[col])+' '
    else:
            words= words+row[col]+' '
    row['bag_of_words']= words
# now these bag of words will help in searching whenever certain words are together then it will show the corresponding row in the search result
data1.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Genre,Director,Actors,Plot,keywords,bag_of_words
Title,,,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]",Two imprisoned men bond over a number of years...,"[two, imprisoned, men, bond, number, years, fi...",crime drama timrobbins morganfreeman bobgunto...
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]",The aging patriarch of an organized crime dyna...,"[aging, patriarch, organized, crime, dynasty, ...",crime drama marlonbrando alpacino jamescaan T...
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]",The early life and career of Vito Corleone in ...,"[early, life, career, vito, corleone, 1920s, n...",crime drama alpacino robertduvall dianekeaton...
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]",When the menace known as the Joker emerges fro...,"[menace, known, joker, emerges, mysterious, pa...",action crime drama christianbale heathledger...
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]",A jury holdout attempts to prevent a miscarria...,"[jury, holdout, attempts, prevent, miscarriage...",crime drama martinbalsam johnfiedler leej.cob...


In [30]:
data1.drop(columns= [col for col in data1.columns if col != 'bag_of_words'], inplace= True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [31]:
data1.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama timrobbins morganfreeman bobgunto...
The Godfather,crime drama marlonbrando alpacino jamescaan T...
The Godfather: Part II,crime drama alpacino robertduvall dianekeaton...
The Dark Knight,action crime drama christianbale heathledger...
12 Angry Men,crime drama martinbalsam johnfiedler leej.cob...


In [32]:
# now count vectorizing it(also called tokenization) 
# It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
count= CountVectorizer()
count_matrix= count.fit_transform(data1['bag_of_words'])

In [33]:
indices= pd.Series(data1.index)
indices[:5]# checking 1st 5 as we cant use .head() on a Series

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [34]:
count.vocabulary_

{'crime': 566,
 'drama': 738,
 'timrobbins': 2530,
 'morganfreeman': 1710,
 'bobgunton': 296,
 'two': 2613,
 'imprisoned': 1212,
 'men': 1640,
 'bond': 300,
 'number': 1795,
 'years': 2793,
 'finding': 933,
 'solace': 2343,
 'eventual': 852,
 'redemption': 2062,
 'acts': 58,
 'common': 501,
 'decency': 627,
 'marlonbrando': 1581,
 'alpacino': 113,
 'jamescaan': 1296,
 'aging': 80,
 'patriarch': 1867,
 'organized': 1827,
 'dynasty': 763,
 'transfers': 2578,
 'control': 535,
 'clandestine': 458,
 'empire': 811,
 'reluctant': 2081,
 'son': 2354,
 'robertduvall': 2145,
 'dianekeaton': 680,
 'early': 765,
 'life': 1489,
 'career': 377,
 'vito': 2701,
 'corleone': 545,
 '1920s': 8,
 'new': 1768,
 'york': 2796,
 'portrayed': 1946,
 'michael': 1656,
 'expands': 872,
 'tightens': 2524,
 'grip': 1080,
 'family': 898,
 'syndicate': 2465,
 'action': 55,
 'christianbale': 442,
 'heathledger': 1125,
 'aaroneckhart': 31,
 'menace': 1641,
 'known': 1432,
 'joker': 1370,
 'emerges': 801,
 'mysterious':

In [35]:
# getting similatitry with help of cosine matrix
cosine_sim= cosine_similarity(count_matrix, count_matrix)

In [36]:
def recommended_movies(title, cosine_sim= cosine_sim):
    recom_movies= []
    idx= indices[indices == title].index[0]
    # creating series based on score in descending order
    score_series= pd.Series(cosine_sim[idx]).sort_values(ascending= False)
    top_10_indexes= list(score_series.iloc[1:11].index)
    # print(top_10_indexes)
    for i in top_10_indexes:
        recom_movies.append(list(data1.index)[i])
    return recom_movies

In [37]:
recommended_movies('Fargo')

['The Departed',
 'The Godfather',
 'Rope',
 'Reservoir Dogs',
 'The Godfather: Part II',
 'Touch of Evil',
 'On the Waterfront',
 'Goodfellas',
 'Arsenic and Old Lace',
 'Cool Hand Luke']

In [38]:
recommended_movies('Rope')

['The Godfather: Part II',
 'Fargo',
 'The Shawshank Redemption',
 'The Godfather',
 'Arsenic and Old Lace',
 'The Silence of the Lambs',
 'Baby Driver',
 'Pulp Fiction',
 'Casino',
 'Strangers on a Train']

In [39]:
# for looking at top actors
data2= imdb_rating[['Title', 'Genre', 'Director', 'Actors', 'Plot']]

In [40]:
data2['Title']= data2['Title'].map(lambda x:x.split(',')[:3]) 
data2['Genre']= data2['Genre'].map(lambda x:x.lower().split(','))
data2['Director']= data2['Director'].map(lambda x:x.split(' '))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [41]:
# colaberative base
market_sugest= pd.read_csv('../input/suggestions/Market_Basket_Optimisation.csv')

In [42]:
market_sugest.head()# here 1st row is the values we dont want it so we need to make it non

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
market= pd.read_csv('../input/suggestions/Market_Basket_Optimisation.csv', header= None)

In [44]:
market.head()# we will use this data set 
# here the header is made as a normal data and has default headers here

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
market.shape

(7501, 20)

In [46]:
transacation= []
for i in range(0, 7501): # here we took the size of data set directly[here 0 to 7501 is row size as we can see above in shape]
    transacation.append([str(market.values[i, j]) for j in range(0, 20)]) # 0 to 20 is columns size

In [ ]:
print(transacation)

In [48]:
# to identify interset of customers we will use in built algorithm call apyori
!pip install apyori

In [50]:
from apyori import apriori

In [72]:
# instentiate the it on data set
model= apriori(transacation, min_support=0.003, min_confidence=0.2, min_lift=3, min_length= 2)
# GIVENING min_length as 2 is returning the combination of which 2 items are bought together

In [73]:
result= list(model)

In [ ]:
print(result) # result is looking shabby

In [77]:
# putting result in a better format so that we can analysis it better
for i in result:
    pair= i[0]
    items= [x for x in pair]
    #print(items)
    print("rule (i.e combination of items)" +  items[0]+ ":->" + items[1])
    print("Support (for that combonation)"+ str(i[1]))
    print("COnfidence (for the combination)"+ str(i[2][0][2]))
    print("Lift"+ str(i[2][0][3]))
    print("====================")

rule (i.e combination of items)light cream:->chicken
Support (for that combonation)0.004532728969470737
COnfidence (for the combination)0.29059829059829057
Lift4.84395061728395
rule (i.e combination of items)escalope:->mushroom cream sauce
Support (for that combonation)0.005732568990801226
COnfidence (for the combination)0.3006993006993007
Lift3.790832696715049
rule (i.e combination of items)escalope:->pasta
Support (for that combonation)0.005865884548726837
COnfidence (for the combination)0.3728813559322034
Lift4.700811850163794
rule (i.e combination of items)fromage blanc:->honey
Support (for that combonation)0.003332888948140248
COnfidence (for the combination)0.2450980392156863
Lift5.164270764485569
rule (i.e combination of items)ground beef:->herb & pepper
Support (for that combonation)0.015997866951073192
COnfidence (for the combination)0.3234501347708895
Lift3.2919938411349285
rule (i.e combination of items)tomato sauce:->ground beef
Support (for that combonation)0.0053326223170